In [1]:
import pandas as pd
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
import sys
import os

sys.path.append(os.path.abspath("../scripts"))

from preProcessing import preprocess_text_column

df_clean = pd.read_csv('../data/fintech_reviews_cleaned.csv')

df_cleaned = preprocess_text_column(df_clean, column="review_text")

from sentiment_analysis import  apply_sentiment_analysis

df = apply_sentiment_analysis(df_cleaned, text_column="review_text")

df.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tsega\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tsega\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\tsega\Desktop\week2-Customer-Experience-Analytics-for-Fintech-Apps\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


,review_text,rating,date,app_name,source,bert_label,bert_score,vader_score,vader_label,textblob_score,textblob_label
0,the app is proactive and a good connections,5,2025-06-05 15:55:10,Commercial Bank of Ethiopia,Google Play,positive,0.999867,0.6908,positive,0.700000,positive
1,i cannot send to cbebirr app through this app,3,2025-06-05 11:12:49,Commercial Bank of Ethiopia,Google Play,negative,0.992965,0.0000,neutral,0.000000,neutral
2,good,4,2025-06-05 10:21:59,Commercial Bank of Ethiopia,Google Play,positive,0.999816,0.4404,positive,0.700000,positive
3,not functional,1,2025-06-05 07:38:12,Commercial Bank of Ethiopia,Google Play,negative,0.999779,0.0000,neutral,0.000000,neutral
4,everytime you uninstall the app you have to re...,1,2025-06-04 21:33:36,Commercial Bank of Ethiopia,Google Play,negative,0.995119,0.0258,neutral,0.196667,positive


In [2]:
# Check if all three models agree on sentiment
df["all_agree"] = (df["bert_label"] == df["vader_label"]) & (df["vader_label"] == df["textblob_label"])

# Check pairwise agreement
df["bert_vader_agree"] = df["bert_label"] == df["vader_label"]
df["bert_textblob_agree"] = df["bert_label"] == df["textblob_label"]
df["vader_textblob_agree"] = df["vader_label"] == df["textblob_label"]

# Summary statistics
agreement_summary = {
    "Total Reviews": len(df),
    "All Agree": df["all_agree"].sum(),
    "BERT & VADER Agree": df["bert_vader_agree"].sum(),
    "BERT & TextBlob Agree": df["bert_textblob_agree"].sum(),
    "VADER & TextBlob Agree": df["vader_textblob_agree"].sum(),
}

# Display summary
for key, value in agreement_summary.items():
    print(f"{key}: {value} ({value / len(df) * 100:.1f}%)")


Total Reviews: 1200 (100.0%)
All Agree: 664 (55.3%)
BERT & VADER Agree: 785 (65.4%)
BERT & TextBlob Agree: 708 (59.0%)
VADER & TextBlob Agree: 978 (81.5%)
